# Step 2. Adding the Daily Returns, Cumulative Returns, Bollinger Bands and Ichimoku Stat Calculations to each stock's csv (will be used for plotting later)

project inspiration from 
https://www.youtube.com/watch?v=4jaBKXDqg9U
    'How to Invest with Data Science' by Derek Banas
  

In [1]:
# import libraries

import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

import datetime as dt
import time

import yfinance as yf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
# make plotly work in jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
from plotly.subplots import make_subplots

from datetime import date
from datetime import timedelta

import warnings
warnings.simplefilter('ignore')

### Save all the data from the CSVs into a dataframe

In [2]:
def get_data_from_csv(ticker):
    try:
        df = pd.read_csv(path + ticker + '.csv', index_col=0)
    except Exception as e:
        print(e)
    
    return df

### Add return columns to dataframes

In [3]:
def add_daily_return_to_df(df):
    '''
    Adds a daily return column to the df
    '''
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    return df


def add_cum_return_to_df(df):
    '''
    Adds cumulative return column to df
    '''
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    return df

### Add Bollinger Bands

In [4]:
def add_bolinger_bands(df):
    '''
    Adds a bolinger band column to the dataframe
    '''
    
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df

### Add Ichimoku Data to Dataframe

The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. Long term traders focus on day or weekly data.

 - Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
 - Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
 - Leading Span A (Senkou A) : Used to identify future areas of support and resistance
 - Leading Span B (Senkou B) : Other line used to identify suture support and resistance
 - Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
 - Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.

In [5]:
def add_Ichimoku(df):
    '''
    Add Ichimoku Data to Dataframe
    '''
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)
    
    # Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    hi_val2 = df['High'].rolling(window=26).max()
    low_val2 = df['Low'].rolling(window=26).min()
    df['Baseline'] = (hi_val2 + low_val2) / 2
    
    # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
    hi_val = df['High'].rolling(window=9).max()
    low_val = df['Low'].rolling(window=9).min()
    df['Conversion'] = (hi_val + low_val) / 2
    
    # Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
    df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)
    
    # Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
    hi_val3 = df['High'].rolling(window=52).max()
    low_val3 = df['Low'].rolling(window=52).min()
    df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)

    return df

## Set Variables

In [6]:
# folder to find and resave dataframes
path = 'C://Users//sophi//Documents//Portfolio_Projects//DS_for_Investing//Wilshire_Stocks//'

In [7]:
# get list of tickers in folder
files = [x for x in listdir(path) if isfile(join(path, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers[:5]

['A', 'AA', 'AA', 'AAL', 'AAME']

## Test above functions on 1 file before running on all 3000+ other files

In [8]:
try: 
    ticker = 'AA'
    print('Working on file', ticker)
    new_df = get_data_from_csv(ticker)
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bolinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
    new_df.to_csv(path + ticker + '.csv')
    print('Success: File', ticker, 'ammended and saved')
    display(new_df.head())
except Exception as e:
    print(e)

Working on file AA
Success: File AA ammended and saved


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Lagging,Baseline,Conversion,SpanA,SpanB
Date,,,,,,,,,,,,,,,,,
2017-07-03,32.775674,34.128890,32.750801,33.850288,3114300,0.0,0,NaN,NaN,NaN,NaN,NaN,38.427338,NaN,NaN,NaN,NaN
2017-07-05,33.999538,34.084117,33.402533,33.601536,4007000,0.0,0,-0.007349,0.992651,NaN,NaN,NaN,37.591530,NaN,NaN,NaN,NaN
2017-07-06,33.442338,35.203508,33.432385,33.720940,6220900,0.0,0,0.003554,0.996179,NaN,NaN,NaN,37.591530,NaN,NaN,NaN,NaN
2017-07-07,33.830385,34.198538,33.099051,34.019436,3776600,0.0,0,0.008852,1.004997,NaN,NaN,NaN,37.004475,NaN,NaN,NaN,NaN
2017-07-10,33.740840,34.278147,33.511988,33.969692,3037400,0.0,0,-0.001462,1.003527,NaN,NaN,NaN,37.312931,NaN,NaN,NaN,NaN


In [9]:
# test was a success
# now lets add daily, cumulative, bollinger bands and ichimoku data to the saved csvs

for ticker in tickers:
    try: 
        print('Working on file', ticker)
        new_df = get_data_from_csv(ticker)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bolinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(path + ticker + '.csv')
        print(f'Success: File {ticker} ammended and saved')
#         display(new_df.head())
    except Exception as e:
        print(e)

Working on file A
Success: File A ammended and saved
Working on file AA
Success: File AA ammended and saved
Working on file AA
Success: File AA ammended and saved
Working on file AAL
Success: File AAL ammended and saved
Working on file AAME
Success: File AAME ammended and saved
Working on file AAN
Success: File AAN ammended and saved
Working on file AAOI
Success: File AAOI ammended and saved
Working on file AAON
Success: File AAON ammended and saved
Working on file AAP
Success: File AAP ammended and saved
Working on file AAPL
Success: File AAPL ammended and saved
Working on file AAT
Success: File AAT ammended and saved
Working on file AAWW
Success: File AAWW ammended and saved
Working on file AAXN
Success: File AAXN ammended and saved
Working on file ABBV
Success: File ABBV ammended and saved
Working on file ABC
Success: File ABC ammended and saved
Working on file ABCB
Success: File ABCB ammended and saved
Working on file ABEO
Success: File ABEO ammended and saved
Working on file ABG
S

Success: File ALNA ammended and saved
Working on file ALNY
Success: File ALNY ammended and saved
Working on file ALOT
Success: File ALOT ammended and saved
Working on file ALPN
Success: File ALPN ammended and saved
Working on file ALRM
Success: File ALRM ammended and saved
Working on file ALSK
Success: File ALSK ammended and saved
Working on file ALSN
Success: File ALSN ammended and saved
Working on file ALT
Success: File ALT ammended and saved
Working on file ALTM
Success: File ALTM ammended and saved
Working on file ALTR
Success: File ALTR ammended and saved
Working on file ALV
Success: File ALV ammended and saved
Working on file ALX
Success: File ALX ammended and saved
Working on file ALXN
Success: File ALXN ammended and saved
Working on file AM
Success: File AM ammended and saved
Working on file AMAG
Success: File AMAG ammended and saved
Working on file AMAL
Success: File AMAL ammended and saved
Working on file AMAT
Success: File AMAT ammended and saved
Working on file AMBC
Success

Success: File ATO ammended and saved
Working on file ATOS
Success: File ATOS ammended and saved
Working on file ATR
Success: File ATR ammended and saved
Working on file ATRA
Success: File ATRA ammended and saved
Working on file ATRC
Success: File ATRC ammended and saved
Working on file ATRI
Success: File ATRI ammended and saved
Working on file ATRO
Success: File ATRO ammended and saved
Working on file ATRS
Success: File ATRS ammended and saved
Working on file ATSG
Success: File ATSG ammended and saved
Working on file ATUS
Success: File ATUS ammended and saved
Working on file ATVI
Success: File ATVI ammended and saved
Working on file AUBN
Success: File AUBN ammended and saved
Working on file AUMN
Success: File AUMN ammended and saved
Working on file AUTO
Success: File AUTO ammended and saved
Working on file AVA
Success: File AVA ammended and saved
Working on file AVAV
Success: File AVAV ammended and saved
Working on file AVB
Success: File AVB ammended and saved
Working on file AVD
Succe

Success: File BMY ammended and saved
Working on file BNED
Success: File BNED ammended and saved
Working on file BNFT
Success: File BNFT ammended and saved
Working on file BOCH
Success: File BOCH ammended and saved
Working on file BOH
Success: File BOH ammended and saved
Working on file BOKF
Success: File BOKF ammended and saved
Working on file BOMN
Success: File BOMN ammended and saved
Working on file BOOM
Success: File BOOM ammended and saved
Working on file BOOT
Success: File BOOT ammended and saved
Working on file BOTJ
Success: File BOTJ ammended and saved
Working on file BOX
Success: File BOX ammended and saved
Working on file BPFH
Success: File BPFH ammended and saved
Working on file BPMC
Success: File BPMC ammended and saved
Working on file BPOP
Success: File BPOP ammended and saved
Working on file BPRN
Success: File BPRN ammended and saved
Working on file BPTH
Success: File BPTH ammended and saved
Working on file BR
Success: File BR ammended and saved
Working on file BRC
Success

Success: File CETV ammended and saved
Working on file CETX
Success: File CETX ammended and saved
Working on file CEVA
Success: File CEVA ammended and saved
Working on file CF
Success: File CF ammended and saved
Working on file CFBK
Success: File CFBK ammended and saved
Working on file CFFI
Success: File CFFI ammended and saved
Working on file CFFN
Success: File CFFN ammended and saved
Working on file CFG
Success: File CFG ammended and saved
Working on file CFMS
Success: File CFMS ammended and saved
Working on file CFR
Success: File CFR ammended and saved
Working on file CFX
Success: File CFX ammended and saved
Working on file CGIX
Success: File CGIX ammended and saved
Working on file CGNX
Success: File CGNX ammended and saved
Working on file CHAP
Success: File CHAP ammended and saved
Working on file CHCI
Success: File CHCI ammended and saved
Working on file CHCO
Success: File CHCO ammended and saved
Working on file CHCT
Success: File CHCT ammended and saved
Working on file CHD
Success:

...

Success: File SCWX ammended and saved
Working on file SCX
Success: File SCX ammended and saved
Working on file SCYX
Success: File SCYX ammended and saved
Working on file SD
Success: File SD ammended and saved
Working on file SDPI
Success: File SDPI ammended and saved
Working on file SEAC
Success: File SEAC ammended and saved
Working on file SEAS
Success: File SEAS ammended and saved
Working on file SEB
Success: File SEB ammended and saved
Working on file SEDG
Success: File SEDG ammended and saved
Working on file SEE
Success: File SEE ammended and saved
Working on file SEIC
Success: File SEIC ammended and saved
Working on file SELB
Success: File SELB ammended and saved
Working on file SELF
Success: File SELF ammended and saved
Working on file SEM
Success: File SEM ammended and saved
Working on file SENEA
Success: File SENEA ammended and saved
Working on file SENEB
Success: File SENEB ammended and saved
Working on file SENS
Success: File SENS ammended and saved
Working on file SERV
Succe

Success: File SRT ammended and saved
Working on file SRTS
Success: File SRTS ammended and saved
Working on file SSB
Success: File SSB ammended and saved
Working on file SSBI
Success: File SSBI ammended and saved
Working on file SSD
Success: File SSD ammended and saved
Working on file SSKN
Success: File SSKN ammended and saved
Working on file SSNC
Success: File SSNC ammended and saved
Working on file SSP
Success: File SSP ammended and saved
Working on file SSTI
Success: File SSTI ammended and saved
Working on file SSTK
Success: File SSTK ammended and saved
Working on file ST
Success: File ST ammended and saved
Working on file STAA
Success: File STAA ammended and saved
Working on file STAG
Success: File STAG ammended and saved
Working on file STAR
Success: File STAR ammended and saved
Working on file STAY
Success: File STAY ammended and saved
Working on file STBA
Success: File STBA ammended and saved
Working on file STC
Success: File STC ammended and saved
Working on file STCN
Success: F

Success: File TMDX ammended and saved
Working on file TMHC
Success: File TMHC ammended and saved
Working on file TMO
Success: File TMO ammended and saved
Working on file TMP
Success: File TMP ammended and saved
Working on file TMST
Success: File TMST ammended and saved
Working on file TMUS
Success: File TMUS ammended and saved
Working on file TNAV
Success: File TNAV ammended and saved
Working on file TNC
Success: File TNC ammended and saved
Working on file TNDM
Success: File TNDM ammended and saved
Working on file TNET
Success: File TNET ammended and saved
Working on file TNXP
Success: File TNXP ammended and saved
Working on file TOL
Success: File TOL ammended and saved
Working on file TORC
Success: File TORC ammended and saved
Working on file TOWN
Success: File TOWN ammended and saved
Working on file TPB
Success: File TPB ammended and saved
Working on file TPC
Success: File TPC ammended and saved
Working on file TPH
Success: File TPH ammended and saved
Working on file TPHS
Success: Fi

Success: File USAP ammended and saved
Working on file USAT
Success: File USAT ammended and saved
Working on file USB
Success: File USB ammended and saved
Working on file USCR
Success: File USCR ammended and saved
Working on file USEG
Success: File USEG ammended and saved
Working on file USFD
Success: File USFD ammended and saved
Working on file USIO
Success: File USIO ammended and saved
Working on file USLM
Success: File USLM ammended and saved
Working on file USM
Success: File USM ammended and saved
Working on file USNA
Success: File USNA ammended and saved
Working on file USPH
Success: File USPH ammended and saved
Working on file USWS
Success: File USWS ammended and saved
Working on file USX
Success: File USX ammended and saved
Working on file UTHR
Success: File UTHR ammended and saved
Working on file UTI
Success: File UTI ammended and saved
Working on file UTL
Success: File UTL ammended and saved
Working on file UTMD
Success: File UTMD ammended and saved
Working on file UUU
Success:

Success: File WLDN ammended and saved
Working on file WLFC
Success: File WLFC ammended and saved
Working on file WLH
Success: File WLH ammended and saved
Working on file WLK
Success: File WLK ammended and saved
Working on file WLL
Success: File WLL ammended and saved
Working on file WLTW
Success: File WLTW ammended and saved
Working on file WM
Success: File WM ammended and saved
Working on file WMB
Success: File WMB ammended and saved
Working on file WMC
Success: File WMC ammended and saved
Working on file WMGI
Success: File WMGI ammended and saved
Working on file WMK
Success: File WMK ammended and saved
Working on file WMS
Success: File WMS ammended and saved
Working on file WMT
Success: File WMT ammended and saved
Working on file WNC
Success: File WNC ammended and saved
Working on file WNEB
Success: File WNEB ammended and saved
Working on file WOR
Success: File WOR ammended and saved
Working on file WOW
Success: File WOW ammended and saved
Working on file WPC
Success: File WPC ammend

In [10]:
print(ticker) # looking at final ticker in list 
new_df.head()

ZYXI


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Lagging,Baseline,Conversion,SpanA,SpanB
Date,,,,,,,,,,,,,,,,,
2017-07-03,0.525223,0.542729,0.498961,0.533976,10340,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.656528,NaN,NaN,NaN,NaN
2017-07-05,0.533976,0.542729,0.533976,0.542729,16280,0.0,0.0,0.016394,1.016394,NaN,NaN,NaN,0.691543,NaN,NaN,NaN,NaN
2017-07-06,0.560237,0.647774,0.542729,0.630266,75680,0.0,0.0,0.161290,1.180328,NaN,NaN,NaN,0.709051,NaN,NaN,NaN,NaN
2017-07-07,0.621514,0.630266,0.612760,0.621514,27830,0.0,0.0,-0.013888,1.163936,NaN,NaN,NaN,0.962908,NaN,NaN,NaN,NaN
2017-07-10,0.621514,0.639020,0.612760,0.639020,18260,0.0,0.0,0.028168,1.196722,NaN,NaN,NaN,0.954154,NaN,NaN,NaN,NaN
